In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
import torch
import numpy as np
import random
import torch.nn as nn
from tqdm import tqdm
random.seed(5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PADDING_CHAR = 0
UNGUESS_CHAR = 27
EMBEDDING_DIM = 128
MAX_LENGTH = 25  # Maximum word length
EPOCHS = 10
EMBEDDING_SIZE = 128

class HangmanModel(nn.Module):
    def __init__(self):
        super(HangmanModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=28, embedding_dim=EMBEDDING_SIZE, padding_idx=0)
        self.bilstm = nn.LSTM(input_size=EMBEDDING_SIZE, hidden_size=256, num_layers=4, dropout=0.01, batch_first=True, bidirectional=True)
        self.fcbilstm = nn.Sequential(
            nn.Linear(256 * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.01)
        )
        self.flags_dense = nn.Sequential(
            nn.Linear(26, 32),
            nn.ReLU(),
            nn.Dropout(0.01)
        )
        self.combined_dense = nn.Sequential(
            nn.Linear(64 + 32, 26)
        )

    def forward(self, word, flags):
        embedded = self.embedding(word)
        bilstm_out, _ = self.bilstm(embedded)
        lastbilstm_out = bilstm_out[:, -1, :]
        bilstm_fcout = self.fcbilstm(lastbilstm_out)
        flags_out = self.flags_dense(flags)

        
        combined = torch.cat((bilstm_fcout, flags_out), dim=1)
        return self.combined_dense(combined)
    
model = HangmanModel()
model.load_state_dict(torch.load("model_BiLSTM3.pth", weights_only=True, map_location=torch.device('cpu'))) # chose model generated after 3rd epoch as later iterations were overfitted

class HangmanPredictor:
    def __init__(self, model):
        """Initialize the predictor with a trained PyTorch model."""
        self.model = model
        print(f"Model loaded: {self.model is not None}")
        self.model.eval()  # Set to evaluation mode
        self.max_length = 25
        self.pad_idx = 0
        
        # Create letter to index mapping
        self.letter_to_idx_map = {'_': 27}
        self.letter_to_idx_map.update({chr(i): i-96 for i in range(97, 123)})

    def letter_to_idx(self, letter):
        """Convert a letter to its corresponding index."""
        return self.letter_to_idx_map.get(letter.lower(), self.pad_idx)

    def convert_to_one_hot(self, guessed_letters):
        """Convert guessed letters to one-hot encoding."""
        guessed_vector = [0] * 26
        for letter in guessed_letters:
            if isinstance(letter, int) and 0 <= letter < 26:
                guessed_vector[letter] = 1
        return guessed_vector

    def pad_sequence(self, sequence):
        """
        Pad or truncate a sequence to a fixed length.
        
        Args:
            sequence (list): The sequence to pad (list of characters).
        
        Returns:
            torch.Tensor: The padded sequence.
        """
        # Convert letters to indices
        numeric_sequence = [self.letter_to_idx(char) for char in sequence]
        
        # Pad or truncate sequence
        if len(numeric_sequence) < self.max_length:
            numeric_sequence.extend([self.pad_idx] * (self.max_length - len(numeric_sequence)))
        else:
            numeric_sequence = numeric_sequence[:self.max_length]
        
        return torch.tensor(numeric_sequence, dtype=torch.long).to(device)

    def guess_next_letter(self, word, guessed_letters):
       
        # Clean and prepare the word
        clean_word = word.replace(" ", "")
        numeric_word = self.pad_sequence(list(clean_word)).unsqueeze(0)  # Add batch dimension
        flags = torch.tensor([[0] * 26], dtype=torch.float32)
        for idx in guessed_letters:
            flags[0,idx] = 1
        # print(f"Input shape: {numeric_word.shape}")
        # Get model predictions
        with torch.no_grad():
            predictions = self.model(numeric_word, flags).cpu().numpy()[0]
        
        # Create mask for already guessed letters
        mask = np.ones(len(predictions), dtype=bool)
        for idx in guessed_letters:
            if 0 <= idx < 26:
                mask[idx] = False
        
        # Find the highest probability unguessed letter
        max_index = np.argmax(np.where(mask, predictions, -np.inf))
        predicted_letter = chr(max_index + 97)
        
        return predicted_letter

    def simulate_game(self, word):
        """
        Simulate a game of Hangman for the given word.
        
        Args:
            word (str): The word to guess
            
        Returns:
            bool: True if the word was guessed successfully, False otherwise
        """
        pattern = "_ " * len(word)
        word_set = set(word.lower())
        all_guesses = set()  # Will store indices (0-25)
        incorrect_guesses = set()
        lives = 6
        correct_guesses = set()

        while "_" in pattern and lives > 0:
            guess = self.guess_next_letter(pattern, all_guesses)
            guess_idx = ord(guess) - 97

            if guess_idx in all_guesses:
                lives -= 1
            else:
                all_guesses.add(guess_idx)

                if guess in word_set:
                    correct_guesses.add(guess)
                    pattern = "".join([char + ' ' if char in correct_guesses else "_ " for char in word.lower()])
                else:
                    incorrect_guesses.add(guess)
                    lives -= 1

        return "_" not in pattern

    def sim_test(self, word_list):
        """
        Test the model on a list of words and return the success rate.
        
        Args:
            word_list (list): List of words to test
            
        Returns:
            float: Success rate (0-1)
        """
        success_rate = []
        for word in tqdm(word_list, desc="Simulating games"):
            success = self.simulate_game(word)
            success_rate.append(success)
        return np.count_nonzero(success_rate)/len(word_list)


# Example usage:
predictor = HangmanPredictor(model)  # Use your PyTorch model path
with open("cleaned_word_list.txt", "r") as file:  # Replace word list with your own
    word_list = file.read().splitlines()

# success = predictor.sim_test(word_list[57000:58000])
# print(f"Success rate: {success:.2f}")


Using device: cpu
Model loaded: True


In [3]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = set()
        
    
    @staticmethod
    def determine_hangman_url():
        links = []                          #links for api call

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions

        guess_letter = predictor.guess_next_letter(word, self.guessed_letters)
        self.guessed_letters.add(ord(guess_letter)-ord('a'))
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    # def build_dictionary(self, dictionary_file_location):
    #     text_file = open(dictionary_file_location,"r")
    #     full_dictionary = text_file.read().splitlines()
    #     text_file.close()
    #     return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = set()
        # self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                # self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [4]:
api = HangmanAPI(access_token="", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [5]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status()
total_runs_tillnow, total_wins_tillnow = total_practice_runs, total_practice_successes
for _ in range(1000):
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = (total_practice_successes - total_wins_tillnow) / (total_practice_runs - total_runs_tillnow)
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs - total_runs_tillnow, practice_success_rate))


run 1000 practice games out of an allotted 100,000. practice success rate so far = 0.674


In [6]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print(practice_success_rate,total_practice_successes, total_practice_runs)

0.6612074502247912 2059 3114


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [9]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game


HangmanAPIError: {'error': 'Your account has been deactivated!'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [10]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print(total_recorded_runs, total_recorded_successes)
print('overall success rate = %.3f' % success_rate)

1000 656
overall success rate = 0.656
